In [35]:
import pandas as pd

# file path to excel file for analysis
filePath = '/Users/adityabehal/Documents/RPI/Sophomore/Summer/UCSF-Nelson-Lab-Research/ASAP-data/3-weeks/Analysis/A1-A3-B4-B0-B2-B5-C2-D1-D2-3-weeks-open-field-analysis.xlsx'

# the three example spreadsheets are binned over one minute
# A1-A3-B4-B0-B2-B5-C2-D1-D2-3-weeks-open-field-analysis parameters
numTrials = 3
arenasPerTrial = 3
animalLabels = ['A1', 'A3', 'B4', 'B0', 'B2', 'B5', 'C2', 'D1', 'D2']
# describes at what minute all mice in a trial have been put into the cylinders
# for example, there are 3 trials for this spreadsheet so 2, 1, and 2 refer to the first, second, and third trial respectively
miceDeployedStartTime = [2, 1, 2]
# Note that most of the time trialOffset will be 0
# If you wanted to exclude trials 1 and 2, for example, and start with trial 3 and onwards for analysis
# then you would set trialOffset to 2 here
trialOffset = 0

'''
# C3-3-weeks-open-field-analysis parameters
numTrials = 1
arenasPerTrial = 1
animalLabels = ['C3']
miceDeployedStartTime = [0]
# Note that most of the time trialOffset will be 0
trialOffset = 0
'''

'''
# E0-E5-F0-F2-F4-F5-G1-G5-G6-H2-H6-H7-3-weeks-open-field-analysis.xlsx parameters
numTrials = 4
arenasPerTrial = 3
animalLabels = ['E0', 'E5', 'F0', 'F2', 'F4', 'F5', 'G1', 'G5', 'G6', 'H2', 'H6', 'H7']
miceDeployedStartTime = [2, 1, 1, 1]
# Note that most of the time trialOffset will be 0
trialOffset = 3
'''

# how long the habituation period is
delayMins = 3
# over how many minutes of data to take the average for the metrics
rangeMins = 10
metrics = ['Velocity', 'Distance moved', 'Rotation', 'Rotation 2']

# actual start time that factors in both miceDeployedStartTime and delayMins
actualStartTime = []

for i in range(len(miceDeployedStartTime)):
    actualStartTime.append(miceDeployedStartTime[i] + delayMins)

# read data from excel spreadsheet into pandas dataframe
dfInput = pd.read_excel(filePath)

# renaming some column names in the spreadsheet
if arenasPerTrial == 1:
    dfInput.rename(columns = {'Unnamed: 1': 'Trials', 'Unnamed: 2': 'Time'}, inplace = True)
else:
    dfInput.rename(columns = {'Unnamed: 1': 'Trials', 'Unnamed: 2': 'Arenas', 'Unnamed: 3': 'Time'}, inplace = True)
    
# print out the first 5 rows in the dataframe
dfInput.head()

,Unnamed: 0,Trials,Arenas,Time,Independent Variable,Independent Variable.1,Independent Variable.2,Independent Variable.3,Velocity,Distance moved,Rotation,Rotation 2,Mobility state,Mobility state.1,Mobility state.2,Mobility state.3,Mobility state.4,Mobility state.5
0,NaN,NaN,NaN,NaN,Independent Variable,Independent Variable,Independent Variable,Independent Variable,Center-point,Center-point,Center-point / Clockwise,Center-point / Counter clockwise,Highly mobile,Highly mobile,Mobile,Mobile,Immobile,Immobile
1,NaN,NaN,NaN,NaN,Animal ID,Familiar object,Novel Object,Treatment,Mean,Total,Frequency,Frequency,Frequency,Cumulative Duration,Frequency,Cumulative Duration,Frequency,Cumulative Duration
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm/s,cm,NaN,NaN,NaN,s,NaN,s,NaN,s
3,Result 1,Trial 1,Arena 1,Start-0:01:00,1,object 2,object 1,Control,5.45985,316.978,3,1,0,0,49,14.7634,50,45.1368
4,Result 1,Trial 1,Arena 1,0:01:00-0:02:00,1,object 2,object 1,Control,3.66946,212.669,1,3,0,0,37,8.17975,37,51.7689


In [36]:
# initialize data of lists.
averageVelocities = []
averageDistances = []
averageRotations = []
averageRotations2 = []

# looping over the trials and arenas per trial to calculate the average metrics for each mouse
for i in range(numTrials):
    actualStartTimeFormatted = "0:0{x}:00-0:0{y}:00".format(x=actualStartTime[i], y=actualStartTime[i] + 1)
    trialFormatted = "Trial     {numTrial}".format(numTrial=i+1+trialOffset)
    for i in range(arenasPerTrial):
        # grabbing the row index corresponding to the first row we are interested
        # we filter the dataframe based on the trial and arena number and the time
        if arenasPerTrial != 1:
            arenaFormatted = "Arena {numArena}".format(numArena=i+1)
            rowStartIndex = dfInput.index[(dfInput["Time"] == actualStartTimeFormatted) & (dfInput["Trials"] == trialFormatted) & (dfInput['Arenas'] == arenaFormatted)].tolist()[0]
        else:
            rowStartIndex = dfInput.index[(dfInput["Time"] == actualStartTimeFormatted) & (dfInput["Trials"] == trialFormatted)].tolist()[0]
        # grabbing the multiple rows that we are interested in (depends on rangeMins value) to calculate average metrics
        dfInputFiltered = dfInput.loc[rowStartIndex:rowStartIndex + rangeMins - 1]
        # you can print out these rows to check that the script is grabbing the rows you are interested in 
        # just uncomment the line below
        # display(dfInputFiltered)
        averageVelocities.append(dfInputFiltered["Velocity"].mean())
        averageDistances.append(dfInputFiltered["Distance moved"].mean())
        averageRotations.append(dfInputFiltered["Rotation"].mean())
        averageRotations2.append(dfInputFiltered["Rotation 2"].mean())

# creating a Python dictionary with the data and saving it to a Pandas dataframe 
data = {'Animal': animalLabels, 'Average velocity': averageVelocities, 'Average distance moved': averageDistances, 'Average rotation': averageRotations, 'Average rotation 2': averageRotations2}

dfOutput = pd.DataFrame(data)
display(dfOutput)

# appending the pandas dataframe to the excel file by creating a new excel sheet in that file called 'Metrics'
# and inserting the pandas dataframe with the animal labels and average metrics into the newly created 'Metrics' excel sheet
with pd.ExcelWriter(filePath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
    dfOutput.to_excel(writer, sheet_name='Metrics', index=False)

,Animal,Average velocity,Average distance moved,Average rotation,Average rotation 2
0,A1,1.859328,106.25265,0.7,0.3
1,A3,2.349174,137.39295,0.9,0.6
2,B4,1.528568,89.19080,0.7,0.4
3,B0,2.872124,172.31922,1.0,1.0
4,B2,2.453134,147.18107,1.5,0.5
5,B5,1.852893,111.16819,1.0,0.9
6,C2,1.786229,107.16860,1.0,0.2
7,D1,4.051734,243.09240,2.5,1.0
8,D2,1.908217,114.48758,0.7,0.4
